In [3]:
import pandas as pd
import numpy as np
import matplotlib.patches as mpatches
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy import stats
import seaborn as sns; sns.set_theme(color_codes=True)
import os
import matplotlib.pyplot as plt
import sklearn.cluster
import sklearn.metrics
import sklearn.datasets
import warnings
warnings.filterwarnings("ignore")
import scipy

In [4]:
import matplotlib
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
sns.set_style(style='white')

In [2]:
common_TADs = pd.read_csv("Carter Lab Datasets\common_TADs_meQTLs.csv")
common_TADs

,Unnamed: 0,chr1,x1,x2,chr2,y1,y2,color,f1,f2,...,H3K27me3,H4K20me1,#meqtl,KMeans_State_Labels,#meqtl_pos,#meqtl_neg,domain_size,norm_meqtl_by_size,norm_pos_meqtl_by_size,norm_neg_meqtl_by_size
0,0,1,36960000,37910000,1,36960000,37910000,"255,255,0",1.8249,0.079690,...,0.167622,0.705304,206,4,71,141,950000,0.062006,0.021654,0.047834
1,1,1,38485000,39310000,1,38485000,39310000,"255,255,0",1.7513,0.051417,...,0.226065,0.369001,219,4,122,97,825000,0.075906,0.042846,0.037893
2,2,1,157140000,157480000,1,157140000,157480000,"255,255,0",1.3679,0.065425,...,0.275085,0.000000,283,4,90,194,340000,0.238009,0.076695,0.183891
3,3,1,64075000,64730000,1,64075000,64730000,"255,255,0",1.3300,0.074093,...,0.244738,0.000000,301,4,201,189,655000,0.131405,0.088911,0.092995
4,4,1,110325000,110530000,1,110325000,110530000,"255,255,0",1.5128,0.032783,...,0.499037,0.704529,38,0,26,22,205000,0.053005,0.036747,0.034587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,1095,X,117750000,117955000,X,117750000,117955000,"255,255,0",1.2678,0.032125,...,0.000000,0.000000,0,0,0,0,205000,0.000000,0.000000,0.000000
1096,1096,X,37785000,38035000,X,37785000,38035000,"255,255,0",1.5301,0.051187,...,1.000000,0.000000,0,1,0,0,250000,0.000000,0.000000,0.000000
1097,1097,X,109150000,109385000,X,109150000,109385000,"255,255,0",1.0945,0.040047,...,0.000000,1.000000,0,1,0,0,235000,0.000000,0.000000,0.000000
1098,1098,X,102790000,102945000,X,102790000,102945000,"255,255,0",1.2129,0.041055,...,0.000000,0.000000,0,1,0,0,155000,0.000000,0.000000,0.000000


In [3]:
meqtls = pd.read_csv("Carter Lab Datasets/meQTL Data/all_cancers_meQTLs.csv")
meqtls

,Unnamed: 0,cancer_type,snp,snp_position,alleles,probes,probe_position,probe_gene,beta,status,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,chr,bp
0,0,BLCA,rs11684598,chr2:33952621,G/A,cg04131969,chr2:33951647,MYADML,-1.24,-40.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr2,33952621
1,1,BLCA,rs12232965,chr2:33954560,C/T,cg04131969,chr2:33951647,MYADML,-1.24,-39.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr2,33954560
2,2,BLCA,rs7574695,chr2:33953186,C/T,cg04131969,chr2:33951647,MYADML,-1.24,-39.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr2,33953186
3,3,BLCA,rs11777332,chr8:91676709,G/A,cg16814680,chr8:91681699,NaN,-1.20,-36.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr8,91676709
4,4,BLCA,rs4332092,chr8:91677926,G/A,cg16814680,chr8:91681699,NaN,-1.20,-36.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr8,91677926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11616294,293448,UCEC,rs1601870,chr3:14024162,T/C,cg23070574,chr3:14187308,XPC,0.42,4.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr3,14024162
11616295,293449,UCEC,rs10903612,chr10:2018353,C/T,cg16296438,chr10:1416569,ADARB2,0.36,4.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr10,2018353
11616296,293450,UCEC,rs4880953,chr10:2019373,C/T,cg16296438,chr10:1416569,ADARB2,0.36,4.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr10,2019373
11616297,293451,UCEC,rs11002432,chr10:79931884,C/T,cg15849445,chr10:80635973,NaN,0.58,4.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr10,79931884


In [42]:
TADs = []
for i, row in meqtls.iterrows():
    if i % 1000 == 0:
        print(i)
    common_chr = common_TADs["chr"+common_TADs["chr1"] == row["chr"]]
    overlap_TAD = common_chr[(row["bp"] >= common_chr["x1"]) & (row["bp"] <= common_chr["x2"])]
    if len(overlap_TAD) > 0:
        if overlap_TAD.iloc[0]["KMeans_State_Labels"] == 0:
            TADs.append("Mixed")
        elif overlap_TAD.iloc[0]["KMeans_State_Labels"] == 1 or overlap_TAD.iloc[0]["KMeans_State_Labels"] == 4:
            TADs.append("Inactive")
        else:
            TADs.append("Active")
    else:
        TADs.append("Boundary")
meqtls["TAD Type"] = TADs
meqtls

MemoryError: Unable to allocate 3.12 GiB for an array with shape (36, 11616299) and data type object

In [4]:
promoter_regions = pd.read_csv("Carter Lab Datasets/Promoter Data/promoter_regions.txt", sep='\t', comment='t', header=None)
header = ["chr", "start", "end", "col3", "col4", "strand"]
promoter_regions.columns = header[:len(promoter_regions.columns)]
promoter_regions.head()

,chr,start,end,col3,col4,strand
0,chr1,10873.0,11873.0,uc001aaa.3_up_1000_chr1_10874_f,0.0,+
1,chr1,10873.0,11873.0,uc010nxr.1_up_1000_chr1_10874_f,0.0,+
2,chr1,10873.0,11873.0,uc010nxq.1_up_1000_chr1_10874_f,0.0,+
3,chr1,16765.0,17765.0,uc009vis.3_up_1000_chr1_16766_r,0.0,-
4,chr1,17751.0,18751.0,uc009vjc.1_up_1000_chr1_17752_r,0.0,-


In [23]:
driver_genes = pd.read_csv("Carter Lab Datasets/Cancer Driver Genes/cancer_driver_genes.csv")
driver_genes

,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),Tumour Types(Germline),Cancer Syndrome,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms
0,A1CF,APOBEC1 complementation factor,29974.0,10:50799421-50885675,2,NaN,11.23,yes,NaN,melanoma,NaN,NaN,E,NaN,oncogene,Mis,NaN,NaN,NaN,"29974,A1CF,ACF,ACF64,ACF65,APOBEC1CF,ASP,ENSG0..."
1,ABI1,abl-interactor 1,10006.0,10:26746593-26860935,1,Yes,12.10,yes,NaN,AML,NaN,NaN,L,Dom,"TSG, fusion",T,KMT2A,NaN,NaN,"10006,ABI-1,ABI1,E3B1,ENSG00000136754.17,Q8IZP..."
2,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25.0,9:130713946-130885683,1,Yes,34.12,yes,NaN,"CML, ALL, T-ALL",NaN,NaN,L,Dom,"oncogene, fusion","T, Mis","BCR, ETV6, NUP214",NaN,NaN,"25,ABL,ABL1,ENSG00000097007.17,JTK7,P00519,c-A..."
3,ABL2,"c-abl oncogene 2, non-receptor tyrosine kinase",27.0,1:179099327-179229601,1,NaN,25.20,yes,NaN,AML,NaN,NaN,L,Dom,"oncogene, fusion",T,ETV6,NaN,NaN,"27,ABL2,ABLL,ARG,ENSG00000143322.19,P42684"
4,ACKR3,atypical chemokine receptor 3,57007.0,2:236569641-236582358,1,Yes,37.30,yes,NaN,lipoma,NaN,NaN,M,Dom,"oncogene, fusion",T,HMGA2,NaN,NaN,"57007,ACKR3,CMKOR1,CXCR7,ENSG00000144476.5,GPR..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,ZNF429,zinc finger protein 429,353088.0,19:21505564-21538078,2,NaN,12.00,yes,NaN,GBM,NaN,NaN,O,NaN,NaN,Mis,NaN,NaN,NaN,"353088,ENSG00000197013.9,Q86V71,ZNF429"
719,ZNF479,zinc finger protein 479,90827.0,7:57119614-57139864,2,NaN,11.20,yes,NaN,"lung cancer, bladder carcinoma, prostate carci...",NaN,NaN,E,NaN,NaN,Mis,NaN,NaN,NaN,"90827,ENSG00000185177.12,KR19,Q96JC4,ZNF479"
720,ZNF521,zinc finger protein 521,25925.0,18:25061926-25352152,1,NaN,11.20,yes,NaN,ALL,NaN,NaN,L,Dom,"oncogene, fusion",T,PAX5,NaN,NaN,"25925,EHZF,ENSG00000198795.10,Evi3,Q96K83,ZNF521"
721,ZNRF3,zinc and ring finger 3,84133.0,22:28883592-29057487,2,NaN,12.10,yes,NaN,"colorectal cancer, adrenocortical carcinoma, g...",NaN,NaN,E,NaN,TSG,"N, F, Mis",NaN,NaN,NaN,"84133,BK747E2.3,ENSG00000183579.15,FLJ22057,KI..."


In [24]:
print(np.array(driver_genes["Role in Cancer"]))

['oncogene' 'TSG, fusion' 'oncogene, fusion' 'oncogene, fusion'
 'oncogene, fusion' 'fusion' 'fusion' 'oncogene' 'TSG' 'oncogene, fusion'
 'fusion' 'oncogene, fusion' 'oncogene, fusion' 'fusion' 'oncogene'
 'oncogene' 'oncogene' 'fusion' 'oncogene, fusion' 'TSG' nan 'TSG'
 'oncogene, TSG' 'oncogene' 'oncogene' 'TSG, fusion' 'oncogene' 'TSG'
 'TSG' 'TSG, fusion' 'TSG, fusion' 'TSG' 'TSG' 'oncogene, TSG, fusion'
 'fusion' 'TSG' 'TSG' 'oncogene, fusion' 'fusion' 'TSG' 'oncogene, TSG'
 'TSG' 'TSG' 'TSG' 'TSG' 'TSG' 'TSG' 'TSG' 'TSG' 'TSG' 'TSG' 'TSG, fusion'
 'oncogene, fusion' 'oncogene, TSG, fusion' 'oncogene, fusion' 'oncogene'
 'oncogene, fusion' 'oncogene, fusion' 'fusion' 'oncogene, fusion'
 'oncogene, TSG' nan 'TSG, fusion' 'oncogene, TSG' 'fusion'
 'oncogene, TSG, fusion' 'oncogene, fusion' 'TSG' nan 'oncogene, TSG'
 'oncogene, fusion' 'TSG' 'TSG' 'oncogene, fusion' 'oncogene, fusion'
 'TSG' 'TSG, fusion' 'oncogene, TSG' 'TSG' 'fusion' 'oncogene' 'oncogene'
 'TSG, fusion' 'fusion' 

In [25]:
driver_genes = driver_genes.dropna(subset = ["Role in Cancer"])
driver_genes

,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),Tumour Types(Germline),Cancer Syndrome,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms
0,A1CF,APOBEC1 complementation factor,29974.0,10:50799421-50885675,2,NaN,11.23,yes,NaN,melanoma,NaN,NaN,E,NaN,oncogene,Mis,NaN,NaN,NaN,"29974,A1CF,ACF,ACF64,ACF65,APOBEC1CF,ASP,ENSG0..."
1,ABI1,abl-interactor 1,10006.0,10:26746593-26860935,1,Yes,12.10,yes,NaN,AML,NaN,NaN,L,Dom,"TSG, fusion",T,KMT2A,NaN,NaN,"10006,ABI-1,ABI1,E3B1,ENSG00000136754.17,Q8IZP..."
2,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25.0,9:130713946-130885683,1,Yes,34.12,yes,NaN,"CML, ALL, T-ALL",NaN,NaN,L,Dom,"oncogene, fusion","T, Mis","BCR, ETV6, NUP214",NaN,NaN,"25,ABL,ABL1,ENSG00000097007.17,JTK7,P00519,c-A..."
3,ABL2,"c-abl oncogene 2, non-receptor tyrosine kinase",27.0,1:179099327-179229601,1,NaN,25.20,yes,NaN,AML,NaN,NaN,L,Dom,"oncogene, fusion",T,ETV6,NaN,NaN,"27,ABL2,ABLL,ARG,ENSG00000143322.19,P42684"
4,ACKR3,atypical chemokine receptor 3,57007.0,2:236569641-236582358,1,Yes,37.30,yes,NaN,lipoma,NaN,NaN,M,Dom,"oncogene, fusion",T,HMGA2,NaN,NaN,"57007,ACKR3,CMKOR1,CXCR7,ENSG00000144476.5,GPR..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716,ZNF331,zinc finger protein 331,55422.0,19:53521024-53580269,1,NaN,13.42,yes,NaN,follicular thyroid adenoma,NaN,NaN,E,Dom,"TSG, fusion",T,?,NaN,NaN,"55422,ENSG00000130844.16,Q9NQX6,RITA,ZNF331,ZN..."
717,ZNF384,zinc finger protein 384 (CIZ/NMP4),171017.0,12:6666648-6689510,1,NaN,13.31,yes,NaN,ALL,NaN,NaN,L,Dom,fusion,T,"EWSR1, TAF15",NaN,NaN,"171017,CAGH1A,CIZ,ENSG00000126746.17,NMP4,NP,Q..."
720,ZNF521,zinc finger protein 521,25925.0,18:25061926-25352152,1,NaN,11.20,yes,NaN,ALL,NaN,NaN,L,Dom,"oncogene, fusion",T,PAX5,NaN,NaN,"25925,EHZF,ENSG00000198795.10,Evi3,Q96K83,ZNF521"
721,ZNRF3,zinc and ring finger 3,84133.0,22:28883592-29057487,2,NaN,12.10,yes,NaN,"colorectal cancer, adrenocortical carcinoma, g...",NaN,NaN,E,NaN,TSG,"N, F, Mis",NaN,NaN,NaN,"84133,BK747E2.3,ENSG00000183579.15,FLJ22057,KI..."


In [26]:
oncogenes = []
tsg = []
other = []
for i, row in driver_genes.iterrows():
    role = row["Role in Cancer"].split(" ")
    for rol in role:
        if rol == "oncogene":
            oncogenes.append(row["Gene Symbol"])
            break
        elif rol == "TSG":
            tsg.append(row["Gene Symbol"])
        else:
            other.append(row["Gene Symbol"])
print(oncogenes)
print(tsg)
print(other)

['A1CF', 'ACVR1', 'AKT1', 'AKT2', 'AKT3', 'AR', 'ARAF', 'ARHGAP5', 'BCL2L12', 'CACNA1D', 'CALR', 'CARD11', 'CCNE1', 'CCR4', 'CCR7', 'CD28', 'CD79A', 'CD79B', 'CDH17', 'CDK4', 'CHD4', 'CSF1R', 'CSF3R', 'CTNNA2', 'CTNND2', 'CXCR4', 'CYSLTR2', 'DDR2', 'DGCR8', 'EGFR', 'ERBB3', 'FGFR4', 'FLT3', 'FLT4', 'FOXA1', 'FUBP1', 'GATA2', 'GNA11', 'GNAQ', 'GNAS', 'GRM3', 'H3F3A', 'H3F3B', 'HIF1A', 'HIST1H3B', 'HRAS', 'IDH1', 'IDH2', 'IKBKB', 'IL6ST', 'IL7R', 'JAK3', 'JUN', 'KAT7', 'KCNJ5', 'KDR', 'KIT', 'KNSTRN', 'KRAS', 'MACC1', 'MAP2K1', 'MAP2K2', 'MAPK1', 'MDM2', 'MDM4', 'MET', 'MITF', 'MPL', 'MTOR', 'MUC16', 'MUC4', 'MYCL', 'MYCN', 'MYD88', 'MYOD1', 'NRAS', 'NT5C2', 'PIK3CA', 'PIK3CB', 'PPM1D', 'PREX2', 'PRKACA', 'PTPN11', 'RAC1', 'REL', 'SALL4', 'SETDB1', 'SF3B1', 'SGK1', 'SIX1', 'SIX2', 'SKI', 'SMO', 'SOX2', 'SRC', 'SRSF2', 'STAT3', 'TNC', 'TRRAP', 'TSHR', 'U2AF1', 'UBR5', 'USP8', 'WAS', 'XPO1', 'ZEB1']
['ACVR2A', 'AMER1', 'APC', 'APOBEC3B', 'ARHGEF10', 'ARHGEF10L', 'ARID1B', 'ARID2', 'ASXL1',

In [27]:
print(len(oncogenes), len(tsg), len(other))

106 223 664


In [28]:
with open('oncogenes.txt', 'w') as f:
    for item in oncogenes:
        f.write("%s\n" % item)

In [29]:
with open('tsg.txt', 'w') as f2:
    for item in tsg:
        f.write("%s\n" % item)

In [38]:
oncogenes_promoters = pd.read_csv("Carter Lab Datasets/oncogenes_promoters.txt", delimiter = "\t", header = None)
oncogenes_promoters.columns = ["chr", "start", "end", "id", "4", "strand"]
oncogenes_promoters = oncogenes_promoters.drop_duplicates(subset=["start", "end"], keep = "last")
oncogenes_promoters.reset_index(drop=True, inplace=True)
oncogenes_promoters

,chr,start,end,id,4,strand
0,chr1,244006886,244007886,NM_001206729.2_up_1000_chr1_244006887_r,0,-
1,chr1,162600764,162601764,NM_001354982.2_up_1000_chr1_162600765_f,0,+
2,chr1,162599652,162600652,NM_001354983.2_up_1000_chr1_162599653_f,0,+
3,chr1,150897738,150898738,NM_001366417.1_up_1000_chr1_150897739_f,0,+
4,chr1,244014381,244015381,NM_001370074.1_up_1000_chr1_244014382_r,0,-
...,...,...,...,...,...,...
167,chr21,44527680,44528680,NM_006758.3_up_1000_chr21_44527681_r,0,-
168,chr21,44527688,44528688,NM_001025203.1_up_1000_chr21_44527689_r,0,-
169,chr22,22221935,22222935,NM_002745.5_up_1000_chr22_22221936_r,0,-
170,chr22,20066763,20067763,NM_022720.7_up_1000_chr22_20066764_f,0,+


In [40]:
tsg_promoters = pd.read_csv("Carter Lab Datasets/tsg_promoters.txt", delimiter = "\t", header = None)
tsg_promoters.columns = ["chr", "start", "end", "id", "4", "strand"]
tsg_promoters = tsg_promoters.drop_duplicates(subset=["start", "end"], keep = "last")
tsg_promoters.reset_index(drop=True, inplace=True)
tsg_promoters

,chr,start,end,id,4,strand
0,chr1,241683055,241684055,NM_000143.4_up_1000_chr1_241683056_r,0,-
1,chr1,116914865,116915865,NM_000701.8_up_1000_chr1_116914866_f,0,+
2,chr1,93296596,93297596,NM_000969.5_up_1000_chr1_93296597_f,0,+
3,chr1,14074909,14075909,NM_001007257.3_up_1000_chr1_14074910_f,0,+
4,chr1,14025682,14026682,NM_001135610.2_up_1000_chr1_14025683_f,0,+
...,...,...,...,...,...,...
364,chr22,29998544,29999544,NM_016418.5_up_1000_chr22_29998545_f,0,+
365,chr22,29998621,29999621,NR_156186.2_up_1000_chr22_29998622_f,0,+
366,chr22,21335585,21336585,NM_006767.4_up_1000_chr22_21335586_f,0,+
367,chr22,24128152,24129152,NM_001362877.2_up_1000_chr22_24128153_f,0,+


In [ ]:
meQTL_driver_gene_promoter = pd.DataFrame()
snp = []
chr1 = []
bp = []
oncogenes = []
tsgs = []

for i, row in me